In [314]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geopy
import config
import time
from geopy.geocoders import MapBox
from geopy.extra.rate_limiter import RateLimiter
import datadotworld as dw

In [3]:
#Need to stash API key in config.py file
locator = MapBox(api_key=api_key)

In [375]:
##Import business file from datadotworld
results = dw.query('darrenfishell/mainely-businesses', 'SELECT * FROM mainely_business_names').dataframe

In [295]:
janky_biz = pd.read_csv('mismatched-addresses.csv')
janky_fix=[]
#Comma logic
janky_biz['commas'] = janky_biz['Address'].str.count(',')

i=0
for i in range(0,len(janky_biz['Address'])):
    
    split_end=janky_biz['commas'][i]
    
    if split_end < 2:
        split_start=0
        split_end=2
    else:
        split_start=janky_biz['commas'][i]-2
        
    splits=janky_biz['Address'][i].split(',')
    
    f=''.join(splits[split_start:split_end+1]).strip()
    janky_fix.append(f)
    i+=1

janky_fix

['PO BOX 388  RAYMOND ME 04071',
 'RR 13 BOX 1776  HOLDEN ME 04429 9400',
 'SUITE 25 KENNEBUNK ME 04043',
 'P.O. BOX 426 PORTLAND ME 04112 0426',
 '181 MAIN STREET PRESQUE ISLE ME 04769',
 'P.O. BOX 1776  LEWISTON ME 04241 1776',
 '37 BEACH ST. SACO ME 04072',
 'PO BOX 21  BURLINGTON ME 04417',
 'PO BOX 586 PORTLAND ME 04112 0586',
 'BOX 1008  PHILLIPS ME 04966',
 '600 PLEASANT STREET WATERTOWN MA 02472',
 'PO BOX 74  SULLIVAN ME 04664',
 '30 MAIN STREET  BANGOR ME 04401',
 '16 GARLAND RD  LINCOLN MA 01773',
 'BOX 4270  UNION ME 04862',
 'KEY WEST FL 33040',
 '45 MEMORIAL CIRCLE AUGUSTA ME 04330',
 '33 PINE PT RD  SCARBOROUGH ME 04074',
 'BOX 4584  CAMDEN ME 04843',
 'PO BOX 325  AUBURN ME 04212',
 '4 STATE STREET ELLSWORTH ME 04605',
 '325 U.S. ROUTE 1 FALMOUTH ME 04105',
 '8 TERRACE COURT  NORTHPORT NY 11768',
 '4 FOXHILL DRIVE  FOXBORO MA 02035',
 'P.O. BOX 248  MADISON ME 04950 0248',
 '850 MAIN STREET - SUITE 3 WESTBROOK ME 04092',
 'ONE WESTON CT. AUGUSTA ME 04330',
 '79 BEN HALL

In [305]:
janky_biz

,Address,mapbox_loc,Names,Status (group),latitude,longitude,commas
0,"PO BOX 388 , RAYMOND, ME 04071","04071 Raymond Road Northeast, Corydon, Indiana...","MAINELY AGILITY, LLC",Active & Good standing,38.265624,-86.026492,2
1,"RR 13 BOX 1776 , HOLDEN, ME 04429 9400",Montenegro,MAINELY BEES LLC,Active & Good standing,42.766670,19.216670,2
2,"C/O BERGEN & PARKINSON LLC , 62 PORTLAND ROAD,...",NaN,MAINELY DENIM,Active & Good standing,NaN,NaN,4
3,"C/O PERKINS THOMPSON, PA , P.O. BOX 426, PORTL...",NaN,"MAINELY EVENTS, LLC",Active & Good standing,NaN,NaN,4
4,"C/O UMPI SOUTH HALL , 181 MAIN STREET, PRESQUE...",NaN,MAINELY GIRLS,Active & Good standing,NaN,NaN,3
5,"C/O THE BELL FIRM, P.A., P.O. BOX 1776, LEW...",NaN,"MAINELY NU PULSE, LLC",Active & Good standing,NaN,NaN,4
6,"C/O PRESCOTT JAMIESON MURPHY LAW GROUP , 37 BE...",NaN,"MAINELY SOCIAL, INC",Active & Good standing,NaN,NaN,3
7,"PO BOX 21 , BURLINGTON, ME 04417","04417 Burlington Loop Northeast, Salem, Oregon...","MAINELY WATER, LLC",Active & Good standing,44.956592,-122.973897,2
8,"C/O VERRILL DANA LLP , PO BOX 586, PORTLAND, M...",NaN,FOREFRONT MAINELY PARTNERS LLC,Inactive,NaN,NaN,3
9,"RR 2, BOX 1008 , PHILLIPS, ME 04966","1008 Box Avenue, Buffalo, New York 14211, Unit...",MAINE-LY A STITCH IN TIME,Inactive,42.911187,-78.824256,3


In [369]:
#Replace bad addresses in Results table
replace_dict = dict(zip(janky_biz['Address'],janky_fix))
results['address'].replace(replace_dict,regex=True,inplace=True)

In [345]:
#Geocode fixed addresses

i=0
mapbox_loc=[]
latitude=[]
longitude=[]

for x in range(0,len(results)):
    try:
        location=locator.geocode(results['address'][i])
        
        time.sleep(.5)
        
        mapbox_loc.append(location.address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        mapbox_loc.append('NA')
        latitude.append('NA')
        longitude.append('NA')
    i+=1

In [370]:
geo_df = pd.DataFrame(
    {'mapbox_loc':mapbox_loc
     ,'latitude':latitude
     ,'longitude':longitude
    })
geo_df

,mapbox_loc,latitude,longitude
0,"Phillips, Maine 04966, United States",44.86,-70.45
1,"162 US Route 1, Scarborough, Maine 04074, Unit...",43.5718,-70.3869
2,"Portland, Portland, Maine 04101, United States",43.6557,-70.2503
3,"6 City Ctr, Portland, Maine 04101, United States",43.657,-70.2567
4,"South Paris, Maine 04281, United States",44.22,-70.52
...,...,...,...
598,"37 Beach Street, Saco, Maine 04072, United States",43.4999,-70.4413
599,"44 Elm Street, Waterville, Maine 04901, United...",44.5492,-69.6343
600,"440 Swamp Road, Durham, Maine 04222, United St...",43.9671,-70.0668
601,"Portland, Maine 04104, United States",43.6582,-70.2667


In [380]:
geo_results = pd.concat([results,geo_df],axis=1)
geo_results

,names,type,urls,status,owner_org_type,address,owner_or_clerk,filing_date,mapbox_loc,latitude,longitude,location
0,MAINE-LY A STITCH IN TIME,MARK,https://icrs.informe.org/nei-sos-icrs/ICRS?Mar...,EXPIRED,INDIVIDUAL,BOX 1008 PHILLIPS ME 04966,MARY O. WILFORD,1993-06-08,"Phillips, Maine 04966, United States",44.860000,-70.450000,POINT(-70.45 44.86)
1,MAINE-LY ACCOUNTING INCORPORATED,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY SUSPENDED,BUSINESS CORPORATION,"162 U.S. ROUTE 1 BOX 4 , SCARBOROUGH, ME 04074",CRAIG RICHARD TREADWELL,1983-07-01,"162 US Route 1, Scarborough, Maine 04074, Unit...",43.571830,-70.386920,POINT(-70.38692 43.57183)
2,"MAINELY STOVES & FUEL YARD, LLC",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY DISSOLVED,LIMITED LIABILITY COMPANY (DOMESTIC),"PO BOX 665 , OLD ORCHARD BEACH, ME 04064",JOHN M. GROVER,2008-12-11,"Old Orchard Road, Old Orchard Beach, Maine 040...",43.505467,-70.411685,POINT(-70.4116846 43.5054674)
3,"PARDUES MAINE-LY WOOD, INC.",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY SUSPENDED,BUSINESS CORPORATION,"704 RIGGS ROAD , CASCO, ME 04015","DAVID PARDUE, PRESIDENT",1990-03-07,"Riggs Rd, Casco, Maine 04015, United States",43.930206,-70.559013,POINT(-70.559012950958 43.9302060654016)
4,"MAINE-LY GYMNASTICS, INC.",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY DISSOLVED,BUSINESS CORPORATION,"RR 7 BOX 1456 , FARMINGDALE, ME 04344",BECKY B. DOHERTY,2001-06-27,"Farmingdale, Maine 04344, United States",44.270000,-69.800000,POINT(-69.8 44.27)
...,...,...,...,...,...,...,...,...,...,...,...,...
598,MAINELY WOOD,ASSUMED,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,GOOD STANDING,NONPROFIT CORPORATION (T13-B),"P.O. BOX 88 , MACHIAS, ME 04654","THOMAS MICHAUD, JR.",1984-12-27,"North Main Street, Machias, Maine 04654, Unite...",44.727200,-67.515390,POINT(-67.5153901 44.7272003)
599,MAINELY WORDS,MARK,https://icrs.informe.org/nei-sos-icrs/ICRS?Mar...,ACTIVE,INDIVIDUAL,"21 KING ROW, DEER ISLE, ME 04627",ELISABETH M. INGOLDSBY,2007-10-11,"21 King Row, Deer Isle, Maine 04627, United St...",44.223725,-68.676089,POINT(-68.676089 44.223725)
600,"MAINELY WRAPS - PORTLAND, LLC",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY DISSOLVED,LIMITED LIABILITY COMPANY (DOMESTIC),"84 MARGINAL WAY , SUITE 600, PORTLAND, ME 0410...",REID HAYTON-HULL,2014-05-22,"84 Marginal Way, Portland, Maine 04101, United...",43.662376,-70.265254,POINT(-70.265254 43.662376)
601,"MAINELY WRAPS - SCARBOROUGH, LLC",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY DISSOLVED,LIMITED LIABILITY COMPANY (DOMESTIC),"84 MARGINAL WAY , SUITE 600, PORTLAND, ME 0410...",REID HAYTON-HULL,2014-05-22,"84 Marginal Way, Portland, Maine 04101, United...",43.662376,-70.265254,POINT(-70.265254 43.662376)


In [381]:
## WRITE GEOCODED FILE TO DATA.WORLD ##
with dw.open_remote_file('darrenfishell/mainely-businesses', 'mainely-business-names.csv') as w:
    results.to_csv(w, index=False)

In [383]:
##Import business file from datadotworld
agents = dw.query('darrenfishell/mainely-businesses', 'SELECT * FROM maine_registered_agents').dataframe

In [384]:
i=0
mapbox_loc=[]
latitude=[]
longitude=[]
geo_df=[]

for x in range(0,len(agents)):
    try:
        location=locator.geocode(agents['full_address'][i])

        mapbox_loc.append(location.address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        mapbox_loc.append('NA')
        latitude.append('NA')
        longitude.append('NA')
    i+=1
    
geo_df = pd.DataFrame(
    {'mapbox_loc':mapbox_loc
    ,'latitude':latitude
    ,'longitude':longitude
    })

In [385]:
agents = pd.concat([agents,geo_df],axis=1)

In [386]:
## WRITE TO DATA.WORLD ##
with dw.open_remote_file('darrenfishell/mainely-businesses', 'maine-registered-agents.csv') as w:
    agents.to_csv(w, index=False)